In [1]:
import pandas as pd
import numpy as np

In [11]:
### calculate a given mape for a daily df

from sklearn.metrics import mean_absolute_error
from sklearn.metrics import mean_squared_error
from sklearn.metrics import median_absolute_error
from sklearn.metrics import r2_score
from sklearn.metrics import mean_squared_log_error

# functions

def wmape(actual, forecast):
    # we take two series and calculate an output a wmape from it, not to be used in a grouping function
    actual = pd.Series(actual)
    forecast = pd.Series(forecast)

    # make a series called mape
    se_mape = abs(actual-forecast)/actual

    # get a float of the sum of the actual
    ft_actual_sum = actual.sum()

    # get a series of the multiple of the actual & the mape
    se_actual_prod_mape = actual * se_mape

    # summate the prod of the actual and the mape
    ft_actual_prod_mape_sum = se_actual_prod_mape.sum()

    # float: wmape of forecast
    ft_wmape_forecast = ft_actual_prod_mape_sum / ft_actual_sum
    print(ft_wmape_forecast)
    # return a float
    return "WMAPE",ft_wmape_forecast

def mean_absolute_percentage_error(y_true, y_pred):
    return "MAPE", np.mean(np.abs((y_true - y_pred) / y_true)) * 100

def mae(y_true, y_pred):
    return "MAE",mean_absolute_error(y_true, y_pred)

def rmse(y_true, y_pred):
    rms = mean_squared_error(y_true, y_pred, squared=False)
    return "RMSE", rms

def mse(y_true, y_pred):
    mse = mean_squared_error(y_true, y_pred, squared=True)
    return "MSE", mse

def medae(y_true, y_pred):
    return "MEDAE", median_absolute_error(y_true, y_pred)

def r2(y_true, y_pred):
    return "R2",r2_score(y_true, y_pred)

def msle(y_true, y_pred):
    return "MSLE", mean_squared_log_error(y_true, y_pred)

def maape(y_true,y_pred): # https://www.sciencedirect.com/science/article/pii/S0169207016000121
    EPSILON = 1e-10
    return "MAAPE", np.mean(np.arctan(np.abs((y_true - y_pred) / (y_true + EPSILON))))

### from peak challenge
def get_peak_values_from_loadpatterm(loadpattern):
  peakvalues = []
  for i in range (int(len(loadpattern)/24)):
    to_consider = loadpattern[i*24:(i+1)*24]
    peakvalues.append(np.max(to_consider))
  return np.array(peakvalues)

def get_peak_times_from_loadpattern(loadpattern):
    peaktimes = []
    for i in range (int(len(loadpattern)/24)):
        to_consider = loadpattern[i*24:(i+1)*24]
        peaktimes.append(to_consider.tolist().index(np.max(to_consider)))
    to_return = np.array(peaktimes)
    to_return = to_return+1
    return to_return


def shape_score (actual_loadpattern,predicted_loadpattern):
    normed_peaksurroundings_actual = []
    normed_peaksurroundings_pred = []
    for block_index in range (int(len(actual_loadpattern)/24)):
        startindex= block_index*24
        block_actual = actual_loadpattern[startindex:(block_index+1)*24]
        peakvalue_actual = np.max(block_actual)
        peakindex_actual  = np.where(block_actual == peakvalue_actual)[0][0]
        normed_actual = actual_loadpattern/peakvalue_actual

        block_pred = predicted_loadpattern[startindex:(block_index+1)*24]
        peakvalue_pred = np.max(block_pred)
        normed_pred = predicted_loadpattern/peakvalue_pred


        normed_peaksurroundings_actual.extend(normed_actual[startindex + peakindex_actual-2:startindex +peakindex_actual+3])
        normed_peaksurroundings_pred.extend(normed_pred[startindex+peakindex_actual-2: startindex+ peakindex_actual+3])
    normed_peaksurroundings_actual = np.array(normed_peaksurroundings_actual)
    normed_peaksurroundings_pred = np.array(normed_peaksurroundings_pred)
    absolute_errors = abs(normed_peaksurroundings_actual-normed_peaksurroundings_pred)
    return "shape_score", np.sum(absolute_errors)

def peaktime_score(actual_peaktimes, predicted_peaktimes):
    absolute_diffs = abs(actual_peaktimes-predicted_peaktimes)
    total_diffs = 0
    for diff in absolute_diffs:
        if abs(diff)<2:
            total_diffs = total_diffs+diff
        elif abs(diff)<5:
            total_diffs = total_diffs+(diff*2)
        else:
            total_diffs = total_diffs+(10)
    return "peaktime_score", total_diffs


def peak_mape(actual_values, forecasted_values):
  peakvalues_actual = get_peak_values_from_loadpatterm(actual_values)
  predictions_at_peaktime = get_peak_values_from_loadpatterm(forecasted_values)
  return "peak_mape", mean_absolute_percentage_error(peakvalues_actual,predictions_at_peaktime)[1]

def peak_rmse(actual_values, forecasted_values):
  peakvalues_actual = get_peak_values_from_loadpatterm(actual_values)
  predictions_at_peaktime = get_peak_values_from_loadpatterm(forecasted_values)
  return "peak_rmse", rmse(peakvalues_actual,predictions_at_peaktime)[1]


def peaktime_mae(actual_values, forecasted_values):
  peaktimes_actual = get_peak_times_from_loadpattern(actual_values)
  predictions_peaktime = get_peak_times_from_loadpattern(forecasted_values)
  return "peaktime_mae", mean_absolute_error(peaktimes_actual,predictions_peaktime)

def percentage_match(peaktimes_actual, predictions_peaktime):
    if len(peaktimes_actual) != len(predictions_peaktime):
        raise ValueError("Arrays must have the same length")

    count_matching = sum(1 for x, y in zip(peaktimes_actual, predictions_peaktime) if x == y)
    total_elements = len(peaktimes_actual)

    percentage = (count_matching / total_elements) * 100
    return percentage

def peaktime_accuracy(actual_values, forecasted_values):
    peaktimes_actual = get_peak_times_from_loadpattern(actual_values)
    predictions_peaktime = get_peak_times_from_loadpattern(forecasted_values)
    result = percentage_match(peaktimes_actual, predictions_peaktime)

    return "peaktime_accuracy", result

# we also evaluate the metrics for the highest n-decile of true value
def highest_decile_metric(y_true,y_pred,function,decile):
    desired_hits = int(len(y_true)*decile)
    ind_highest = np.argpartition(y_true, -desired_hits)[-desired_hits:]
    top_hits_true = y_true[ind_highest]
    top_hits_pred = y_pred[ind_highest]
    name, res = function(top_hits_true,top_hits_pred)
    return name+"_decile_"+str(decile), res

# evaluation

functions = [mean_absolute_percentage_error,mae,rmse,r2,peak_rmse,peaktime_mae,peaktime_accuracy]

def evaluate_metrics(target_df, functions):
    results = {}
    for function in functions:
        name, res = function(target_df["Agg Load"].values, target_df["Predict"].values)
        results.update({name:res})

    return results

In [36]:
methods = ["LSTM", "XGB","Random Forest"]
nn_methods = ["LSTM"]

runs = 4

hh_results = pd.DataFrame()
hp_results = pd.DataFrame()
total_results = pd.DataFrame()

hh_comparison = pd.DataFrame()
hp_comparison = pd.DataFrame()
total_comparison = pd.DataFrame()


for method in methods: # first, we obtain the results of the different runs 
    
    for run in range(1,runs+1):
        comb = pd.DataFrame()
        hp_df = pd.read_pickle("./HP "+method+" Results "+str(run)+".pkl")
        hh_df = pd.read_pickle("./HH "+method+" Results "+str(run)+".pkl")
        comb_df = pd.read_pickle("./Comb "+method+" Results "+str(run)+".pkl")
        if method in nn_methods:
            hp_df.rename(columns={"NN Forecast":"Predict"},inplace=True)
            hh_df.rename(columns={"NN Forecast":"Predict"},inplace=True)
            comb_df.rename(columns={"NN Forecast":"Predict"},inplace=True)
                
        # creating df for plotting
        #hh_comparison.index = hh_df.index
        #hp_comparison.index = hp_df.index
        total_comparison.index = comb_df.index
        hh_comparison[method+" Run "+str(run)] = hh_df["Predict"].values
        hp_comparison[method+" Run "+str(run)] = hp_df["Predict"].values
        total_comparison[method+" Run "+str(run)] = comb_df["Predict"].values
        hh_comparison["Real Load"] = hh_df["Agg Load"].values
        hp_comparison["Real Load"] = hp_df["Agg Load"].values
        total_comparison["Real Load"] = comb_df["Agg Load"].values

        hh_results[method+" Run "+str(run)] = evaluate_metrics(hh_df,functions)
        hp_results[method+" Run "+str(run)] = evaluate_metrics(hp_df,functions)
        total_results[method+" Run "+str(run)] = evaluate_metrics(comb_df,functions)
            
            


In [37]:
# then we calculate descriptive statistics per method and create an average as ensemble model
df_compare_rmse = pd.DataFrame()
for method in methods:
    filtered_cols = [x for x in total_results.columns if method in x]
    filtered_results_hh = hh_results[filtered_cols]
    filtered_results_hp = hp_results[filtered_cols]
    filtered_results_comb = total_results[filtered_cols]
    
    df_compare_rmse[method+" HH"] = filtered_results_hh.T.describe()["RMSE"]
    df_compare_rmse[method+" HP"] = filtered_results_hp.T.describe()["RMSE"]
    df_compare_rmse[method+" Comb"] = filtered_results_comb.T.describe()["RMSE"]
    
    if method in nn_methods: # build from average forecast final forecast
        temp_hh = hh_df.copy()
        temp_hh["Predict"] = hh_comparison[filtered_cols].mean(axis=1).values
        temp_hh.to_pickle("./../HH "+method+" Results.pkl") 
        
        temp_hp = hp_df.copy()
        temp_hp["Predict"] = hp_comparison[filtered_cols].mean(axis=1).values
        temp_hp.to_pickle("./../HP "+method+" Results.pkl") 
        
        temp_comb = comb_df.copy()
        temp_comb["Predict"] = total_comparison[filtered_cols].mean(axis=1).values
        temp_comb.to_pickle("./../Comb "+method+" Results.pkl") 
        
        


In [38]:
# showing the results for the rmse
df_compare_rmse.T[["mean","std","min","max"]]

,mean,std,min,max
LSTM HH,1452.564509,6.327636,1447.214594,1461.475168
LSTM HP,2989.198882,90.291138,2908.373873,3083.380447
LSTM Comb,3096.271956,58.887180,3014.159976,3152.671079
XGB HH,1362.025453,0.000000,1362.025453,1362.025453
XGB HP,2817.448650,0.000000,2817.448650,2817.448650
XGB Comb,3256.327808,0.000000,3256.327808,3256.327808
Random Forest HH,1368.292017,6.925888,1362.011511,1374.834449
Random Forest HP,2761.412442,4.085598,2756.476246,2765.767370
Random Forest Comb,3256.019219,10.805400,3242.943742,3269.301888
